# EfficientDet

ARXIV [https://arxiv.org/pdf/1911.09070.pdf](https://arxiv.org/pdf/1911.09070.pdf)  
Github [https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch](https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch)  

EfficientDet employs EfficientNet as the backbone network, BiFPN as the feature network, and shared class/box prediction network. Both BiFPN layers and class/box net layers are repeated multiple times based on different resource constraints.

[Object Detection SOTA model](https://paperswithcode.com/sota/object-detection-on-coco)  
This page shows object detection models' score on COCO test-dev. I focus AP50 score, because this competitions' metric is AP50.    
1. DyHead (Based Swin-L) : 78.5
2. DetectoRS (Based ResNeXt) : 74.2
3. YOLOv4-P7 (Based Scaled-YOLO) : 73.3
4. EfficientDet-D7 (Based EfficientNet) : 72.4
5. YOLOv4-608 (Based YOLO) : 65.7


<figure>
<img src="https://blog.roboflow.com/content/images/2020/06/yolov5-performance.png" style="width:700px">
    <figcaption>EfficientDet is better model than YOLOv5 on AP.</figcaption>
</figure>


Everyone used model based YOLOv4 or YOLOv5, but this model isn't SOTA model. I'll try EfficientDet first and then Scaled-YOLOv4, DyHead.  
  
Three notebooks summarize how to use this model.
1. [Preprocessing](https://www.kaggle.com/adldotori/efficientdet-preprocessing-better-than-yolov5/)
2. [Training](https://www.kaggle.com/adldotori/efficientdet-training-better-than-yolov5/)
3. [Inference](https://www.kaggle.com/adldotori/efficientdet-inference-better-than-yolov5/) - 2days later open!

This notebook is first notebook which includes how to preprocess the data.  
Let's start!  

This picture shows the rough structure of efficientdet.
![image](https://aihub-storage.s3.ap-northeast-2.amazonaws.com/file/efficientdet.png)

# Environment

In [ ]:
!conda install gdcm -c conda-forge -y
!pip install pycocotools numpy opencv-python tqdm tensorboard tensorboardX pyyaml webcolors matplotlib

# Base Setting

Clone EfficientDet repository. In this repo, we can training after preprocessing **pre-trained weights**, **yml file**, **image files(not dcm)**, **annotation files**.

In [ ]:
!git clone https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch

import os
os.chdir("Yet-Another-EfficientDet-Pytorch")

In [ ]:
# load checkpoint
! mkdir weights
! wget https://github.com/zylo117/Yet-Another-EfficientDet-Pytorch/releases/download/1.0/efficientdet-d0.pth -O weights/efficientdet-d0.pth

In [ ]:
siim_yml = '''
project_name: siim  # also the folder name of the dataset that under data_path folder
train_set: train
val_set: val
num_gpus: 1

# mean and std in RGB order, actually this part should remain unchanged as long as your dataset is similar to coco.
mean: [ 0.485, 0.456, 0.406 ]
std: [ 0.229, 0.224, 0.225 ]

# this anchor is adapted to the dataset
anchors_scales: '[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)]'
anchors_ratios: '[(1.0, 1.0), (1.3, 0.8), (1.9, 0.5)]'

obj_list: ['typical', 'indeterminate', 'atypical']
'''
with open('projects/siim.yml', 'w') as f:
    f.write(siim_yml)

# Change to 256x256px Image

In [ ]:
import os
from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

import torch

def read_xray(path, voi_lut=False, fix_monochrome=True):
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array

    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data

    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)

    return data


def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)

    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)

    return im

In [ ]:
from glob import glob
INPUT_PATH = "/kaggle/input/siim-covid19-detection/"

for split in ["test", "train"]:
    save_dir = f"datasets/siim/{split}/"

    os.makedirs(save_dir, exist_ok=True)

    for path in tqdm(glob(INPUT_PATH + split + '/*/*/*')):
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(path)
        im = resize(xray, size=256)
        im.save(os.path.join(save_dir, path.split('/')[-1][:-3]+'jpg'))

# Preprocessing

Let's check what the csv file looks like.

In [ ]:
import os
import pandas as pd
from glob import glob
import pydicom

In [ ]:
train_study = pd.read_csv(INPUT_PATH + 'train_study_level.csv')
train_image = pd.read_csv(INPUT_PATH + 'train_image_level.csv')

In [ ]:
train_study.head()

id is too long, and column name too. Let's shorten the name.

In [ ]:
train_study = train_study.rename(columns = {
    'Negative for Pneumonia': 'Negative', 'Typical Appearance': 'Typical',
    'Indeterminate Appearance': 'Indeterminate', 'Atypical Appearance': 'Atypical'},
                                       inplace = False)
train_study['StudyInstanceUID'] = train_study['id'].str[:-6]
train_study.drop(columns=['id'], inplace=True)
train_study.head()

In [ ]:
train_image.head()

This dataframe's id is too long, and merge train_study dataframe.

In [ ]:
train_image = train_image.merge(train_study, on='StudyInstanceUID')
train_image['id'] = train_image['id'].str[:-6]
train_image.head()

Add path of image.

In [ ]:
id_path = []
for i in glob('/kaggle/input/siim-covid19-detection/train/*/*/*'):
    id_path.append((i, i.split('/')[-1][:-4]))
id_path = pd.DataFrame(id_path, columns=['path', 'id'])
train_image = train_image.merge(id_path, on='id')
train_image.head()

In [ ]:
train_image.iloc[0]['boxes']

In [ ]:
train_image.iloc[0]['label']

boxes column includes x, y, width, height.

<img src="https://aihub-storage.s3.ap-northeast-2.amazonaws.com/file/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA_2021-07-08_%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE_8.51.15.png" style="width:500px">

label column includes x1, y1, x2, y2.

<img src="https://aihub-storage.s3.ap-northeast-2.amazonaws.com/file/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA_2021-07-08_%E1%84%8B%E1%85%A9%E1%84%92%E1%85%AE_8.54.13.png" style="width:500px">

In [ ]:
pydicom.read_file(train_image.iloc[0]['path']).pixel_array.shape

We can know the image size in the same way as above.

In [ ]:
xy = []
for i, data in train_image.iterrows():
    xy.append(pydicom.read_file(data['path']).pixel_array.shape)
train_image[['xcell','ycell']] = xy
train_image.to_csv('datasets/train_image.csv', index=None)
train_image.head()

# Split train to train&val

Make the validation set.

In [ ]:
import random
import os
import shutil
import pandas as pd
import json

random.seed(481)
SRC_PATH = 'datasets/siim/train/'
TRG_PATH = 'datasets/siim/'
train_list = os.listdir(SRC_PATH)
random.shuffle(train_list)

import shutil
os.makedirs(TRG_PATH+'val', exist_ok=True)
for path in train_list[int(len(train_list)*0.8):]:
    shutil.move(SRC_PATH + path, TRG_PATH + 'val/' + path)

# Annotation Files

We need categories, images, annotations(box). Let's make this file.

In [ ]:
def anno(sets='train'):
    image_id = pd.DataFrame(os.listdir(TRG_PATH + sets))[0].str[:-4].values.tolist()
    annotation = {}
    annotation['type'] = 'instances'
    annotation['categories'] = []
    annotation['images'] = []
    annotation['annotations'] = []
    annotation['categories'].append({'supercategory': 'none', 'id': 1, 'name': 'typical'})
    annotation['categories'].append({'supercategory': 'none', 'id': 2, 'name': 'indeterminate'})
    annotation['categories'].append({'supercategory': 'none', 'id': 3, 'name': 'atypical'})
    for i, data in train_image[train_image.id.isin(image_id)].iterrows():
        dic = {}
        dic['file_name'] = data['id']+'.jpg'
        dic['height'] = 256
        dic['width'] = 256
        dic['id'] = data.name + 1
        annotation['images'].append(dic)
        cnt = 1

    for i, data in train_image.iterrows():
        if type(data['boxes']) == float: # nan
            continue
        # split box string
        boxes = json.loads(data['boxes'].replace('\'', '\"'))
        
        # reverse x,y cell count
        ycell, xcell = data['xcell'], data['ycell']
        
        # category
        t, i, a = data['Typical'], data['Indeterminate'], data['Atypical']
        if t==1:
            category = 1
        elif i==1:
            category = 2
        elif a == 1:
            category = 3
        
        # add boxes
        for j in boxes:
            dic = {}
            dic['area'] = (j['width']*256)//xcell * (j['height']*256)//ycell
            dic['iscrowd'] = 0
            dic['image_id'] = data.name + 1
            dic['bbox'] = [(j['x']*256)//xcell, (j['y']*256)//ycell,
                        (j['width']*256)//xcell, (j['height']*256)//ycell]
            dic['category_id'] = category
            dic['id'] = cnt
            dic['ignore'] = 0
            dic['segmentation'] = []
            cnt += 1
            annotation['annotations'].append(dic)
            
    # save annotation json files
    with open(f'{TRG_PATH}annotations/instances_{sets}.json', 'w') as f:
        json.dump(annotation, f)

In [ ]:
os.makedirs(TRG_PATH + 'annotations', exist_ok=True)
anno('train')
anno('val')

# Finished!

The whole preprocessing process is complete. I will train on the next notebook with the files from here. Please wait for the next notebook. Thank you for read my notebook!